In [1]:
import pandas as pd

In [2]:
INPUT_FILEPATH = "input/produccion_junio_2023.xlsx"
MES_A_ANALIZAR = "June"

In [27]:

# Este diccionario relaciona la hoja de los procedimientos a datos en el formato de SIGCOM.
# El primer elemento de la lista corresponde a la fila en donde debe ser cargado ese procedimiento.
# El segundo elemento corresonde a que CC de consulta debe ser cargado el procedimiento (en caso
# que tenga alguna consulta)
CORRELACION_PROCEDIMIENTOS_SIGCOM = {
    "QMT": ["PROCEDIMIENTO ONCOLOGÍA", "CONSULTA ONCOLOGÍA"],
    "APNEA": ["PROCEDIMIENTO DE NEUMOLOGÍA", "CONSULTA NEUMOLOGÍA"],
    "EF": ["PROCEDIMIENTOS DE CARDIOLOGÍA", ""],
    "EBUS": ["PROCEDIMIENTOS EBUS", "CONSULTA NEUMOLOGÍA"],
    "TAVI": ["PROCEDIMIENTOS TAVI", ""],
    "ASIST VENTRICULAR": ["PROCEDIMIENTOS ASISTENCIA VENTRICULAR", ""],
    "ECMO": ["PROCEDIMIENTOS ECMO", ""],
    "HMD": ["PROCEDIMIENTOS DE HEMODINAMIA", "CONSULTA CARDIOLOGÍA"],
    "BRONCOSCOPIA": ["PROCEDIMIENTOS ENDOSCÓPICOS", "CONSULTA NEUMOLOGÍA"],
}

SERVICIO_INTERNO_SIGCOM = {
    "ambulatorio": "CONSULTAS",
    "medico quirugico 3° norte y sur": "HOSPITALIZACIÓN MEDICINA INTERNA/HOSPITALIZACIÓN QUIRÚRGICA",
    "medico quirugico 4° norte": "HOSPITALIZACIÓN CARDIOVASCULAR",
    "uci 4° sur": "UNIDAD DE CUIDADOS INTENSIVOS",
    "uci 5° norte": "UNIDAD DE CUIDADOS INTENSIVOS",
    "uci 5° sur": "UNIDAD DE CUIDADOS INTENSIVOS",
    "uti 3° norte": "UNIDAD DE TRATAMIENTO INTENSIVO ADULTO",
    "uti 4° norte": "UNIDAD DE TRATAMIENTO INTENSIVO ADULTO",
    "uti 4° sur": "UNIDAD DE TRATAMIENTO INTENSIVO ADULTO",
    "uti 5° norte": "UNIDAD DE TRATAMIENTO INTENSIVO ADULTO",
}

In [28]:
def obtener_localizacion_de_procedimientos(ruta_produccion):
    resultados = {}
    archivo = pd.ExcelFile(ruta_produccion)
    for hoja in archivo.sheet_names[3:]:
        df = pd.read_excel(archivo, sheet_name=hoja)
        df = df.loc[:, "Rut":"Hoja1.Nombre nuevo HIS"]
        df["Hoja1.Nombre nuevo HIS"] = df["Hoja1.Nombre nuevo HIS"].str.strip().str.lower()
        df["SIGCOM"] = df["Hoja1.Nombre nuevo HIS"].replace(SERVICIO_INTERNO_SIGCOM)

        procedimientos_mes = df[df["Fecha Realización"].dt.month_name() == MES_A_ANALIZAR]
        procedimientos_por_servicio = (
            procedimientos_mes.groupby("SIGCOM")["Rut"]
            .count()
            .reset_index(name="cantidad_procedimientos")
        )

        consulta_acorde = CORRELACION_PROCEDIMIENTOS_SIGCOM[hoja][-1]
        dict_reemplazo_consulta = {"CONSULTAS": consulta_acorde}
        procedimientos_por_servicio["SIGCOM"] = procedimientos_por_servicio["SIGCOM"].replace(
            dict_reemplazo_consulta
        )

        procedimiento_acorde = CORRELACION_PROCEDIMIENTOS_SIGCOM[hoja][0]

        resultados[procedimiento_acorde] = procedimientos_por_servicio

    return resultados

In [29]:
resultados_procedimientos = obtener_localizacion_de_procedimientos(INPUT_FILEPATH)

In [30]:
formato = pd.read_excel("input/Formato_5_Distribución_Producción_CC_Apoyo.xlsx", header=2)
formato = formato.iloc[:, 1:]

In [41]:
def explode_and_multiply(df, column_name):
    """
    Explodes rows with comma-separated values in a given column.

    Parameters:
        df (pd.DataFrame): Input DataFrame.
        column_name (str): The name of the column containing comma-separated values.

    Returns:
        pd.DataFrame: The resulting DataFrame after exploding.
    """
    # First, create a copy of the DataFrame to avoid modifying the original DataFrame
    new_df = df.copy()

    # Split the values in the specified column by comma and explode them into separate rows
    new_df[column_name] = new_df[column_name].str.split('/')
    new_df = new_df.explode(column_name)

    return new_df

In [43]:
explode_and_multiply(resultados_procedimientos["PROCEDIMIENTO DE NEUMOLOGÍA"], "SIGCOM")

,SIGCOM,cantidad_procedimientos
0,CONSULTA NEUMOLOGÍA,65
1,HOSPITALIZACIÓN MEDICINA INTERNA,3
1,HOSPITALIZACIÓN QUIRÚRGICA,3
